---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [22]:
import pandas as pd
import numpy as np
import calendar
import re
doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)


In [47]:
#125 out of 500: only included mm/dd/yy , mm/dd/yyyy , mm-dd-yyyy ,  mm-dd-yy (range:0~124, length:125)
##Accumulated Total:125
df_num = df.str.extractall(r'(?P<MM>\b\d{1,2})[/-](?P<DD>\d{1,2})[/-](?P<YYYY>\d{2,4}\b)')
df_num['dx'] = df_num.index.get_level_values(0)
df_num = df_num.astype(int)
def nin(row):
    if(row<100):
        return row+1900
    else:
        return row
df_num['YYYY'] = df_num['YYYY'].map(nin)
#69 out of 500: only included dd Month yyyy, dd mon yyyy, dd mon. yyyy, dd month, yyyy (range:125~193, length:69)
##Accumulated Total:194
## Drop the first column since the first and second columns are duplicated (they are completely same)
df_eu=df.str.extractall(r'(?P<DD>\s?\d{1,2}[-|.|\s|th|st|nd])\s?((?P<MM>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-|\s|.|,])(?P<YYYY>\s?\d{2,4})').drop(1,1)
df_eu = df_eu[['MM','DD','YYYY']]

def month_to_num(row):
    dicc = {v:k for k,v in enumerate(calendar.month_abbr)}
    return dicc[row]
df_eu['MM'] = df_eu['MM'].map(month_to_num)
df_eu['dx'] = df_eu.index.get_level_values(0)
df_eu = df_eu.astype(int)

#34 out of 500:only included mm-dd-yyyy, mm dd, yyyy, mon dd, yyyy, mm. dd, yyyy, mm dd yyyy (range:194~227, length:34)
##Accumulated Total:228
df_us=df.str.extractall(r'\s?((?P<MM>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-|\s|.|,])(?P<DD>\s?\d{1,2}.{1,2}[,|\s|-])\s?(?P<YYYY>\d{2,4})').drop(0,1)
df_us['DD'] = df_us['DD'].str.replace(r'[,\s]',"")
df_us['MM'] = df_us["MM"].map(month_to_num)
df_us['dx'] = df_us.index.get_level_values(0)
df_us = df_us.astype(int)

#114 out of 500: only included mon yyyy (range:228~342, length:102)
##Accumulated Total:342
df_short_us = df.str.extractall(r'((?P<MM>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\s?(?P<YYYY>\d{4}))').drop(0,1)
##Since there is no day in original dataset, all the day columns are considered the first day of the month 
df_short_us['DD'] = 1
df_short_us = df_short_us[['MM','DD','YYYY']]
df_short_us = df_short_us[(228,0):]
df_short_us['MM'] = df_short_us['MM'].map(month_to_num)
df_short_us['dx'] = df_short_us.index.get_level_values(0)
df_short_us = df_short_us.astype(int)

#112 out of 500: m/yyyy, mm/yyyy (range:343~454, length: 112)
##(A lot of rows from the beginning are duplicated with df_num so subtract the duplicated rows in df_num)
##Accumulated Total:454
df_short_num = df.str.extractall((r'\s?(?P<MM>\d{1,2}/)(?P<DD>)\s?(?P<YYYY>\d{4})'))
##Convert the type of the column 'MM' to integer since in order to confirm the month that must be less than 12
df_short_num['MM'] = df_short_num['MM'].str.replace('/',"").astype(int)
df_short_num = df_short_num[(343,0):]
df_short_num['DD'] = 1
df_short_num = df_short_num.astype(int)

####### not compeletely sure about the accuracy of the result ##########
#45 out of 500: YYYY(range:455~499, length: 45)
## Assume the year without the month, January 1st.
df_year = df.str.extractall(r'(?P<MM>)(?P<DD>)(?P<YYYY>[^\d/-][012]\d{3}|^[012]\d{3})')
df_year['YYYY'] = df_year['YYYY'].str.replace(r'[syr~\(.]',"").astype(int)
df_year[['MM','DD']] = 1
df_year = df_year[(455,0):]
df_year = df_year.astype(int)

for dff in [df_short_num,df_year]:
    dff['dx'] = dff.index.get_level_values(0)
    

In [18]:
df[339]

'Currently on Zoloft, 50 mg, Dicolenac, 100 mg, Hydrocodone, Seroquel, 100 mg. Was hospitalized for Serotonin Syndrome for 3 days in March, 2005 due to Dr. Potts increasing effexor and zoloft. Current Psychopharmacologist(s) and Phone #(s):\n'

In [213]:
#Combine dataframes
df_united = df_num.append([df_eu,df_us,df_short_us,df_short_num,df_year])
df_united.drop(1,level='match',inplace=True,axis=0)
df_united.reset_index(inplace=True)
df_united.drop(['level_0','match'],axis=1,inplace=True)

In [212]:
#missed part fixing process
#Total:14
miss_list = []
for i in range(500):
    if i not in list(df_united['dx']):
        miss_list.append(i)
#print('list of index missed:',miss_list) 

miss_df = df[miss_list]

#regular exp
case1_miss = miss_df.str.extractall(r'[^\d][\s](?P<MM>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[,]\s(?P<DD>)(?P<YYYY>[12]\d{3})')
case2_miss = miss_df.str.extractall(r'^(?P<MM>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[,]\s(?P<DD>)(?P<YYYY>[12]\d{3})')
case3_miss = miss_df.str.extractall(r'(?P<DD>[\d])(?P<MM>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[,]\s(?P<YYYY>[12]\d{3})')
case3_miss = case3_miss[['MM','DD','YYYY']]
case4_miss = miss_df.str.extractall(r'(?P<MM>[0]\d{1,2})[/](?P<DD>\d{2})[/](?P<YYYY>\d{2})').astype(int)

#create index columns, convert month to int. 
for dfs in [case1_miss,case2_miss,case3_miss,case4_miss]:
    dfs['dx'] = dfs.index.get_level_values(0)
    
    if dfs['MM'].dtype != int:
        dfs['MM'] = dfs['MM'].map(month_to_num)
    
    dfs['DD'].replace(np.nan,1,inplace=True)
        
#Since the function 'astype' doesn't work so manually convert the whole dataframe into integer type.
case1_miss = case1_miss.astype(int)
case2_miss = case2_miss.astype(int)
case3_miss = case3_miss.astype(int)
case4_miss = case4_miss.astype(int)

#Change the type 'YY -> 'YYYY'
case4_miss['YYYY'] = case4_miss['YYYY'] + 1900

#combine all the cases, reset the multilevel index to single
miss_united = case1_miss.append([case2_miss,case3_miss,case4_miss])
miss_united.reset_index(inplace=True)
miss_united.drop(['level_0','match'],1,inplace=True)

In [247]:
united = df_united.append(miss_united)
united.reset_index(inplace=True)
united.drop('index',1,inplace=True)
united = united.astype(str)
#united.apply()
def alla(row):
    row['all'] = row['MM']+'/'+row['DD'] +'/'+row['YYYY']
    return row

united = united.apply(alla,axis=1)
united_all = united[['all','dx']]
united_all['all'] = pd.to_datetime(united_all['all'])

#df['Date'] =pd.to_datetime(df.Date)

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [260]:
united_all = united_all.sort_values(by='all')
answer = united_all['dx'].astype(int).reset_index().drop('index',1)
answer


,dx
0,9
1,84
2,2
3,53
4,28
5,474
6,153
7,13
8,129
9,98


In [271]:
def date_sorter():
    import pandas as pd
    import numpy as np
    import calendar
    import re
    doc = []
    with open('dates.txt') as file:
        for line in file:
            doc.append(line)

    df = pd.Series(doc)    
#125 out of 500: only included mm/dd/yy , mm/dd/yyyy , mm-dd-yyyy ,  mm-dd-yy (range:0~124, length:125)
##Accumulated Total:125
    df_num = df.str.extractall(r'(?P<MM>\b\d{1,2})[/-](?P<DD>\d{1,2})[/-](?P<YYYY>\d{2,4}\b)')
    df_num['dx'] = df_num.index.get_level_values(0)
    df_num = df_num.astype(int)
    def nin(row):
        if(row<100):
            return row+1900
        else:
            return row
    df_num['YYYY'] = df_num['YYYY'].map(nin)
#69 out of 500: only included dd Month yyyy, dd mon yyyy, dd mon. yyyy, dd month, yyyy (range:125~193, length:69)
##Accumulated Total:194
## Drop the first column since the first and second columns are duplicated (they are completely same)
    df_eu=df.str.extractall(r'(?P<DD>\s?\d{1,2}[-|.|\s|th|st|nd])\s?((?P<MM>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-|\s|.|,])(?P<YYYY>\s?\d{2,4})').drop(1,1)
    df_eu = df_eu[['MM','DD','YYYY']]

    def month_to_num(row):
        dicc = {v:k for k,v in enumerate(calendar.month_abbr)}
        return dicc[row]
    df_eu['MM'] = df_eu['MM'].map(month_to_num)
    df_eu['dx'] = df_eu.index.get_level_values(0)
    df_eu = df_eu.astype(int)

#34 out of 500:only included mm-dd-yyyy, mm dd, yyyy, mon dd, yyyy, mm. dd, yyyy, mm dd yyyy (range:194~227, length:34)
##Accumulated Total:228
    df_us=df.str.extractall(r'\s?((?P<MM>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-|\s|.|,])(?P<DD>\s?\d{1,2}.{1,2}[,|\s|-])\s?(?P<YYYY>\d{2,4})').drop(0,1)
    df_us['DD'] = df_us['DD'].str.replace(r'[,\s]',"")
    df_us['MM'] = df_us["MM"].map(month_to_num)
    df_us['dx'] = df_us.index.get_level_values(0)
    df_us = df_us.astype(int)

#114 out of 500: only included mon yyyy (range:228~342, length:102)
##Accumulated Total:342
    df_short_us = df.str.extractall(r'((?P<MM>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\s?(?P<YYYY>\d{4}))').drop(0,1)
##Since there is no day in original dataset, all the day columns are considered the first day of the month 
    df_short_us['DD'] = 1
    df_short_us = df_short_us[['MM','DD','YYYY']]
    df_short_us = df_short_us[(228,0):]
    df_short_us['MM'] = df_short_us['MM'].map(month_to_num)
    df_short_us['dx'] = df_short_us.index.get_level_values(0)
    df_short_us = df_short_us.astype(int)

#112 out of 500: m/yyyy, mm/yyyy (range:343~454, length: 112)
##(A lot of rows from the beginning are duplicated with df_num so subtract the duplicated rows in df_num)
##Accumulated Total:454
    df_short_num = df.str.extractall((r'\s?(?P<MM>\d{1,2}/)(?P<DD>)\s?(?P<YYYY>\d{4})'))
##Convert the type of the column 'MM' to integer since in order to confirm the month that must be less than 12
    df_short_num['MM'] = df_short_num['MM'].str.replace('/',"").astype(int)
    df_short_num = df_short_num[(343,0):]
    df_short_num['DD'] = 1
    df_short_num = df_short_num.astype(int)

####### not compeletely sure about the accuracy of the result ##########
#45 out of 500: YYYY(range:455~499, length: 45)
## Assume the year without the month, January 1st.
    df_year = df.str.extractall(r'(?P<MM>)(?P<DD>)(?P<YYYY>[^\d/-][012]\d{3}|^[012]\d{3})')
    df_year['YYYY'] = df_year['YYYY'].str.replace(r'[syr~\(.]',"").astype(int)
    df_year[['MM','DD']] = 1
    df_year = df_year[(455,0):]
    df_year = df_year.astype(int)

    for dff in [df_short_num,df_year]:
        dff['dx'] = dff.index.get_level_values(0)
    
#Combine dataframes
    df_united = df_num.append([df_eu,df_us,df_short_us,df_short_num,df_year])
    df_united.drop(1,level='match',inplace=True,axis=0)
    df_united.reset_index(inplace=True)
    df_united.drop(['level_0','match'],axis=1,inplace=True)
#missed part fixing process
#Total:14
    miss_list = []
    for i in range(500):
        if i not in list(df_united['dx']):
            miss_list.append(i)
    #print('list of index missed:',miss_list) 

    miss_df = df[miss_list]

#regular exp
    case1_miss = miss_df.str.extractall(r'[^\d][\s](?P<MM>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[,]\s(?P<DD>)(?P<YYYY>[12]\d{3})')
    case2_miss = miss_df.str.extractall(r'^(?P<MM>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[,]\s(?P<DD>)(?P<YYYY>[12]\d{3})')
    case3_miss = miss_df.str.extractall(r'(?P<DD>[\d])(?P<MM>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[,]\s(?P<YYYY>[12]\d{3})')
    case3_miss = case3_miss[['MM','DD','YYYY']]
    case4_miss = miss_df.str.extractall(r'(?P<MM>[0]\d{1,2})[/](?P<DD>\d{2})[/](?P<YYYY>\d{2})').astype(int)

#create index columns, convert month to int. 
    for dfs in [case1_miss,case2_miss,case3_miss,case4_miss]:
        dfs['dx'] = dfs.index.get_level_values(0)

        if dfs['MM'].dtype != int:
            dfs['MM'] = dfs['MM'].map(month_to_num)

        dfs['DD'].replace(np.nan,1,inplace=True)

#Since the function 'astype' doesn't work so manually convert the whole dataframe into integer type.
    case1_miss = case1_miss.astype(int)
    case2_miss = case2_miss.astype(int)
    case3_miss = case3_miss.astype(int)
    case4_miss = case4_miss.astype(int)

#Change the type 'YY -> 'YYYY'
    case4_miss['YYYY'] = case4_miss['YYYY'] + 1900

#combine all the cases, reset the multilevel index to single
    miss_united = case1_miss.append([case2_miss,case3_miss,case4_miss])
    miss_united.reset_index(inplace=True)
    miss_united.drop(['level_0','match'],1,inplace=True)
    united = df_united.append(miss_united)
    united.reset_index(inplace=True)
    united.drop('index',1,inplace=True)
    united = united.astype(str)
#combine all the columns and make the full date type 
    def alla(row):
        row['all'] = row['MM']+'/'+row['DD'] +'/'+row['YYYY']
        return row

    united = united.apply(alla,axis=1)

#seperate the two columns for converting the type to datetime in order to sort in ascending order.
    united_all = united[['all','dx']]
    united_all['all'] = pd.to_datetime(united_all['all'])
    united_all = united_all.sort_values(by='all')
    answer = united_all['dx'].astype(int).reset_index().drop('index',1)
    answer

    return answer.iloc[:,0]